In [1]:
using AeroAcoustics
using HDF5
using DelimitedFiles # for data export
using DSP # for Hanning window
using PyPlot # for plotting

In [2]:
time_file = "DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_TimeSeries.h5"
csm_file = "DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_octave-12_CsmEss.h5"

"DTU_PLCT_NACA63018_trip_5PS_5SS_U0_50_AoA_0_octave-12_CsmEss.h5"

## Task 1

In [7]:
# Collect time-series data and microphone geometry
function DTU_AIAAreadTimeseries(destfile::String)
    h5open(destfile, "r") do file
        MicrophoneData = read(file, "MicrophoneData")
        t = MicrophoneData["microphoneDataPa"]
        t = permutedims(t,reverse(1:ndims(t)))
        MetaData = read(file, "MetaData")
        micgeom = MetaData["ArrayAttributes"]["microphonePositionsM"]
        return t,micgeom
    end
    
end
t,micgeom = DTU_AIAAreadTimeseries(time_file);

In [14]:
# Collect meta data
ArrayAttributes = h5readattr(time_file,"MetaData/ArrayAttributes")
MeasurementData = h5readattr(time_file,"MeasurementData")
TestAttributes = h5readattr(time_file,"MetaData/TestAttributes")
MicrophoneData = h5readattr(time_file,"MicrophoneData/microphoneDataPa")

Dict{String, Any} with 2 entries:
  "sampleCount"  => 84
  "sampleRateHz" => 16384

In [20]:
fs = MicrophoneData["sampleRateHz"]
n = 4096
CSM = csm(t;n=n,noverlap=div(n,2),fs=fs,win=DSP.hanning(n),scaling="spectrum");

In [60]:
# Locate indices of fc = 500Hz and 1000Hz:
fc = [500,1000]
f_idx = Int.(fc/(fs/n)).+1
for (fi,fc) in zip(f_idx,fc)
    CSM_out = CSM.arr[:,:,fi]
    writedlm("data/task1_DTU_AeroAcousticsjl_CSM_real_f_$(fc).csv",  real.(CSM_out), ';')
    writedlm("data/task1_DTU_AeroAcousticsjl_CSM_imag_f_$(fc).csv",  imag.(CSM_out), ';')
end


## Task 2

In [61]:
# To compute the psf, an environment must be defined, which requires a cross-spectral matrix.
# This can be instantiated with some fake data.
fc = [500,1000,2000,4000]
CSM = FreqArray(rand(84,84,4),fc)
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = false,
    Nx = 41,
    Ny = 41,
    xlim = (-1,1),
    ylim = (-1,1),
    flim = (500,4000)
);
steeringvectors!(E)
p = psf(E);

1681×4 FreqArray{Float64, 2, Vector{Int64}}:
 0.0110854   0.000620191  0.000739052  0.0180986
 0.0097512   0.00154288   0.00124666   0.00801647
 0.00833005  0.00288905   0.00173099   0.00939586
 0.00689083  0.00442991   0.00202488   0.00979164
 0.00550971  0.00589915   0.00218199   0.00594576
 0.00426578  0.00704082   0.00253651   0.00137886
 0.00323576  0.00765815   0.00345576   0.000262808
 0.00248817  0.00765247   0.00490896   0.000959484
 0.00207747  0.00704216   0.00630797   0.0018869
 0.00203855  0.00595615   0.00690009   0.00647067
 ⋮                                     
 0.00218883  0.00015211   0.00143827   0.00241953
 0.00267113  0.000116614  0.00200863   0.000479908
 0.00348883  0.000106379  0.0029795    0.00149271
 0.00457511  0.000114357  0.00419174   0.00138758
 0.00584891  0.000141319  0.00506501   0.023112
 0.00722228  0.000192292  0.00490171   0.0595317
 0.00860736  0.000271646  0.00345635   0.058692
 0.00992258  0.000378894  0.00137403   0.0288528
 0.0110976   0.00050

In [66]:
for (fc,pif) in zip(fc,eachcol(p))
    writedlm("data/task2_DTU_AeroAcousticsjl_PSF_f_$(fc).csv",  reshape(pif,41,41), ';')
end

## Task 3

In [3]:
# Collect CSM-data and microphone geometry
function DTU_AIAAreadCSM(destfile::String)
    h5open(destfile, "r") do file
        CsmData = read(file, "CsmData")
        fc = CsmData["binCenterFrequenciesHz"]
        CSM = CsmData["csmReal"]+im*CsmData["csmImaginary"]
        CSM = permutedims(CSM,reverse(1:ndims(CSM)))
        MetaData = read(file, "MetaData")
        micgeom = MetaData["ArrayAttributes"]["microphonePositionsM"]
        return FreqArray(CSM,fc),micgeom
    end
    
end
CSM,micgeom = DTU_AIAAreadCSM(csm_file);

In [4]:
# Collect meta data
ArrayAttributes = h5readattr(csm_file,"MetaData/ArrayAttributes")
CsmData = h5readattr(csm_file,"CsmData")
MeasurementData = h5readattr(csm_file,"MeasurementData")
TestAttributes = h5readattr(csm_file,"MetaData/TestAttributes")

Dict{String, Any} with 5 entries:
  "domainBoundsM"       => @NamedTuple{1::Float64, 2::Float64, 3::Float64}[(var…
  "flowType"            => "kevlar"
  "machNumber"          => [0.150235, 0.0, 0.0]
  "coordinateReference" => "array center"
  "testDescription"     => "Airfoil in kevlar-walled wind tunnel"

In [7]:
# Setup environment
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = true,
    Nx = 41,
    Ny = 41,
    xlim = (-2,2),
    ylim = (-1,1),
    flim = (400,4000)
);
steeringvectors!(E)

In [8]:
b = beamforming(E);

In [92]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task3_DTU_AeroAcousticsjl_CBF_f_$(fc).csv",  reshape(SPL.(b[:,fi]),41,41), ';')
end

## Task 4

In [9]:
x = cleanSC(E;ϕ=0.5,maxiter=100);

1681×40 FreqArray{Float64, 2, Vector{Float64}}:
 0.0         0.0         7.57411e-8  …  0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         2.05066e-7     0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0         …  0.0  0.0  7.87814e-8  0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 0.0         0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 1.01773e-7  0.0         0.0            0.0  0.0  0.0         0.0  0.0  0.0
 ⋮                                   ⋱       ⋮                          
 1.27902e-7  1.75e-7     0.0            0.0  0.0  0.0         0.0  0.0  0.0
 3.0089e-7   0.0         7.38984e-7     0.0

In [10]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task4_DTU_AeroAcousticsjl_CleanSC_f_$(fc).csv",  reshape(SPL.(x[:,fi]),41,41), ';')
end

## Task 5

In [11]:
# Setup environment with shear-layer
c = MeasurementData["speedOfSoundMPerS"]
Ma = TestAttributes["machNumber"][1]
E = Environment(
    CSM=CSM,
    micgeom=micgeom,
    z0 = TestAttributes["domainBoundsM"][1][3],
    dr = true,
    shear = true,
    Ma = -Ma,
    c = c,
    h = 1.5,
    Nx = 41,
    Ny = 41,
    xlim = (-2,2),
    ylim = (-1,1),
    flim = (400,4000)
);
steeringvectors!(E)

In [12]:
b = beamforming(E);

In [13]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task5_DTU_AeroAcousticsjl_CBF_f_$(fc).csv",  reshape(SPL.(b[:,fi]),41,41), ';')
end

## Task 6

In [23]:
# Source integration region
limits = [−0.5,0.5,−0.4,0.4]
srcint_cbf = sourceintegration(b,E,limits);
writedlm("data/task6_DTU_AeroAcousticsjl_CBF_srcint.csv",  [E.fn SPL.(srcint_cbf)], ';')

## Task 7

In [24]:
x = cleanSC(E;ϕ=0.5,maxiter=100);

In [25]:
# Locate indices of fc = 500Hz, 1000Hz, 2000Hz, 4000Hz:
fc = [500,1000,2000,4000]
f_idx =  [argmin(abs.(E.fn .- fc)) for fc in fc] # finds index in E.fn that is closest to fc

for (fi,fc) in zip(f_idx,fc)
    writedlm("data/task7_DTU_AeroAcousticsjl_CleanSC_f_$(fc).csv",  reshape(SPL.(x[:,fi]),41,41), ';')
end

## Task 8

In [26]:
# Source integration region
limits = [−0.5,0.5,−0.4,0.4]
srcint_clean = sourceintegration(x,E,limits);
writedlm("data/task8_DTU_AeroAcousticsjl_CleanSC_srcint.csv",  [E.fn SPL.(srcint_clean)], ';')